In [3]:
import numpy as np
from numpy.fft import *
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
from PIL import Image
import os
from datetime import datetime
from IPython.display import Video

In [ ]:
# Meaningful parameters
n = 512
L = 2*np.pi
nu = 1e-5
dt = 1
print("Mesh ratio :", nu*dt*n*n/L/L)
steps = 500
jump = 5

# Initialize te grid
def lico(x, y):
    return x.reshape(len(x), 1) @ y.reshape(1, len(y))
x = L/n*np.arange(n)
X = lico(x, np.ones(n))
Y = lico(np.ones(n), x)

# Initialisation of vorticity
w = np.random.rand(n, n)-0.5
#plt.pcolor(X, Y, w)
#plt.show()

# Precompute everything that is possible !
k = np.arange(n)
k[n//2+1:] -= n
k = (2*np.pi/L*k)**2
K = lico(k, np.ones(n//2+1)) + lico(np.ones(n), k[:n//2+1])
LAP = -K
K[0, 0] = 10e20
INV_LAP = -1 / K
DX = np.zeros(n, dtype=complex)
DX[:n//2] = np.arange(n//2)
DX[n//2+1:] = np.arange(1-n//2, 0)
DX = 2j*np.pi/L * lico(DX, np.ones(n//2+1))
DY = np.arange(n//2+1)
DY[n//2] = 0
DY = 2j*np.pi/L * lico(np.ones(n), DY)

## Colors 
cmap = colors.LinearSegmentedColormap.from_list("test", [[0.6, 0, 0], [1, 0, 0],[1, 1, 0], [0, 1, 1], [0, 0, 1], [0, 0, 6]])
cmap

In [ ]:
os.system("rm -f Images/*.tiff")

In [ ]:
%%time 
for i in range(steps):
    W = rfft2(w)
    PSI = -INV_LAP * W
    u = irfft2(DY * PSI)
    v = irfft2(-DX * PSI)
    k1 = nu*irfft2(LAP*W) - u*irfft2(DX*W) - v*irfft2(DY*W)
     
    W = rfft2(w + dt/2 * k1)
    PSI = -INV_LAP * W
    u = irfft2(DY * PSI)
    v = irfft2(-DX * PSI)
    k2 = nu*irfft2(LAP*W) - u*irfft2(DX*W) - v*irfft2(DY*W)
    
    W = rfft2(w + dt/2 * k2)
    PSI = -INV_LAP * W
    u = irfft2(DY * PSI)
    v = irfft2(-DX * PSI)
    k3 = nu*irfft2(LAP*W) - u*irfft2(DX*W) - v*irfft2(DY*W)
    
    W = rfft2(w + dt * k3)
    PSI = -INV_LAP * W
    u = irfft2(DY * PSI)
    v = irfft2(-DX * PSI)
    k4 = nu*irfft2(LAP*W) - u*irfft2(DX*W) - v*irfft2(DY*W)
    
    w += dt/6 * (k1 + 2*k2 + 2*k3 + k4)
    
    if i % jump == 0:
        min_w = min(w.flat)
        max_w = max(w.flat)
        norm = (w - min_w)/ (max_w - min_w)
        im = Image.fromarray(cmap(norm, bytes=True))
        im.save("Images/" + str(i//jump).zfill(5) + ".tiff")

In [ ]:
dt_string = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")
os.system("ffmpeg -hide_banner -loglevel error -framerate 24 -i Images/%05d.tiff -pix_fmt yuv420p -c:v libx264 -preset slow -crf 18 Videos/" + dt_string + ".mp4")

# Good parameters :

$n=512, L=2\pi, \nu=10^{-5}, dt=0,2$

In [4]:
import pyfftw
import multiprocessing

pyfftw.config.NUM_THREADS = 6
n = 4096
a = pyfftw.empty_aligned((n, n), dtype='float')
b = pyfftw.empty_aligned((n, n//2+1), dtype='complex')

a = np.random.rand(n, n)

fft_object_c = pyfftw.FFTW(a, b, axes=(0,1))

In [ ]:
%%timeit
fft_object_c()

In [ ]:
%%timeit
c = fft(a)